# Lab 2 (Part D) - Nonlinear regression

<div class="alert alert-block alert-danger">

__IMPORTANT__ 
Please complete this Jupyter Notebook file and upload it to blackboard __before 05 February 2020__.
</div>

In this part, you will implement a nonlinear kernel regression. We will use the same house pricing dataset as previously. The following code simply loads the dataset from the data file into the variables $X$ and $y$:

In [ ]:
# Initialization, always run this cell before anything else
%matplotlib notebook

In [ ]:
import numpy as np

filename = "datasets/ex1data2.txt"
mydata = np.genfromtxt(filename, delimiter=",")

# We have n data-points
n = len(mydata)

# X is a matrix of two column, i.e. an array of n 2-dimensional data-points
X = mydata[:, :2].reshape(n, 2)

# y is the vector of outputs, i.e. an array of n scalar values
y = mydata[:, -1]

""" TODO:
print a subset of X and y to see how it looks like
"""
# ...

You are asked to implement the *Nadaraya-Watson estimator*, which consists in a kernel regression using the *Gaussian kernel* function.

The Gaussian kernel function of two vectors $v$ and $u$ is defined as $k(u, v) = e^{{-\left \| u - v \right \|}^2 ~/~ 2 \sigma^2}$, where $\sigma$ (sigma) is a hyperparameter representing the width of the Gaussian. The equation of the Gaussian kernel can be simplified to $k(u, v) = e^{{- \gamma ~ \left \| u - v \right \|}^2}$ by considering a hyperparameter gamma: $\gamma = \frac{1}{\sigma^2}$. Complete the code below to write the Gaussian kernel function `gaussian_kernel(..)` corresponding to $k(u, v) = e^{{- \gamma ~ \left \| u - v \right \|}^2}$. You can use the Python function `math.exp(..)` to compute the exponancial, and `np.linalg.norm(u - v)` (or your own function) to compute the distance belween two vectors (numpy arrays) $u$ and $v$.

The hypothesis function $h(x)$ to make a prediction about a new test data-point $x$ (i.e. predict the price of a new house), is defined as follows:
$$h(x) = \frac{1}{\sum_{i=1}^n k(x, x^{(i)})} \sum_{i=1}^n k(x, x^{(i)}) ~ y^{(i)},$$
where $x$ is the test data-point, $x^{(i)}$ is the $i^{th}$ training data-point, and $y^{(i)}$ is the output (price) corresponding to the $i^{th}$ training data-point. Complete the code below to write `h(..)` the hypothesis function (which calls the `gaussian_kernel(..)` function).

Once the hypothesis function `h(..)` is implemented, use it with $\gamma = 0.00005$ to make a price prediction for a new house of 1650-square-foot with 3 bedrooms.

In [ ]:
import math
import numpy as np

""" TODO: 
Write the definition of the gaussian_kernel(...) function. It takes as 
arguments two vectors u and v, and a hyperparameter gamma. This function 
can be considered as a measure of similarity between u and v.
"""
def gaussian_kernel(u, v, gamma):
    ...
    return ...


""" TODO: 
Write the code for the hypothesis function h. The arguments are:
*** x: one new test data-point
*** X: the training data
*** y: the training outputs
*** gamma: the hyperparameter of the gaussian kernel
Note: be careful about devisions by zero.
"""
def h(x, X, y, gamma):
    # An array containing the similarity between x and all the others data-points in X :
    similarities = np.array([ gaussian_kernel(x, xi, gamma) for xi in X ])
    ...
    return ...


""" TODO:
Make a prediction for a new house x of 1650-square-foot with 3 bedrooms using gamma = 0.00005
"""
gamma = 0.00005
x = np.array([1650, 3])
# prediction = ...
# print("The prediction on x is:", prediction)


In order to see that the hyperparameter $\gamma$ affects the predicted output, you are asked to vary $\gamma$ between $1e-10$ and $10e-5$ (you can use `np.arange(1e-10, 10e-5, 1e-5)` in Python), and see how the price predicted for the house `x = np.array([1650, 3])` varies with $\gamma$. Do a plot of the $\gamma$ values with respect to the price predicted for house `x`.

**Note**: We will see later in the course how to automatically select a good value for hyperparameters such as $\gamma$, using cross-validation.

In [ ]:
import matplotlib.pylab as plt

""" TODO:
Complete the following code to predict the price of x based on different values of gamma
"""

x = np.array([1650, 3])
gammas_list = np.arange(1e-10, 10e-5, 1e-5)
predictions = []

for gamma in gammas_list:
    pass
    # prediction = ... make a prediction on x using the current gamma ...
    # append the prediction to the list of predictions

fig, ax = plt.subplots()
# TODO: plot the values the predicted prices of x with respect to the corresponding values of gamma
# ...
fig.show()


Finally, we split the training dataset into two parts. The first part `X_train` (and `y_train`) is used as a training data. The second part `X_test` is used to make price predictions and compare it to the true prices `y_test`.

Complete the code to produce a list `y_pred` which contains the predicted price of each house in `X_test`. Then, produce a scatter plot based on `X_test`, `y_pred` and `y_test`, which shows the first feature (house size) on one axis vs. the predicted and true price on the other axis. The plot should look like the Figure below.

Try several values of $\gamma$ (e.g. $0.5, 0.00005, 0.00000001$) and see each time on the plot how your predicted prices change. Notice that when $\gamma$ is smaller, the predicted values tend to be similar regradless of the test houses. Why ? Think about it before looking at the answer below.

*Answer*: When $\gamma$ is set to a very small value, the Gaussian width $\sigma$ is very large. This makes all the values $\{ k(x, x^{(i)}) \mid i=1 \dots n \}$ large (close to 1), i.e., all the training data-points will equaly influence $h(x)$  (the predicted price of $x$). Therefore, the predicted price of any test point $x$ will just be the average price of all training houses.

<img src="imgs/scatterLab2D.png" style="width: 500px;"/>

In [ ]:
gamma = 0.00005 # 0.5 0.00005, 0.00000001
n = len(X)

# The training part:
X_train = X[ : n//2] # First half of the dataset
y_train = y[ : n//2] # Outputs corresponding to the first half

# The testing part:
X_test = X[n//2 : ] # Second half of the dataset
y_test = y[n//2 : ] # Outputs corresponding to the second half


""" TODO: 
Based on X_train and y_train, predict the price of each house in X_test. 
These predictions will be in a list named y_pred.
"""
# y_pred = ...


""" TODO:
Complete the following code to produce a figure similar to the one shown above.
"""
fig, ax = plt.subplots()
# TODO: scatter plot of the first feature (column 0) of X_test vs. y_test
# TODO: scatter plot of the first feature (column 0) of X_test vs. y_pred
# ...
# ...
# ...
fig.show()
